<img src="figures/lwx.png" alt="Logo LWX" title="" style="height:100px" align="right"/>
<img src="figures/logo_IFL.svg" alt="Logo IFL" title="" style="height:100px" align="left"/>

## Entwicklung eines automatisierten Auslegungs- und Optimierungsprogramms für anisotrope UAV-Propeller

### Lukas Hilbers, E-Mail: l.hi@posteo.de
---

## Agenda
- #### Motivation
- #### Herangehensweise
- #### Software-Architektur
- #### Lastenrechnung mit XFOIL und XROTOR
- #### Fe-Modellierung mit PyMAPDL
---

## Motivation
<img src="figures/Helics_Mat_1.jpg" alt="Naturfasern" title="" style="height:400px" align="right"/>

- #### Studienarbeit: Evaluierung naturfaserverstärkter Kunststoffe für Einsatz in Multicopter-Propellern
    * #### FE-Simulationen mit generischem Modell in ANSYS-CLASSIC
    * #### Werkstoffe (insbes. FLAXPREG) prinzipiell geeignet
    * #### Offene Themen: Lastrechnung und Übertragung auf reale Geometrien
    * #### Workflow: Weitgehend händisch, umständlich
- #### Interessengebiete: Optimierung und Programmierung
- #### Idee: Entwicklung einer automatisierten Toolchain zur Auslegung in Python
    * #### PyMAPDL: interaktives Interface zwischen Python und Ansys
    * #### PyOpt: wissenschaftlich fundierte Optimierungs-Toolbox

---

## Lastrechnung

<img src="figures/xfoil_logo.gif" alt="Naturfasern" title="" style="height:100px" align="right"/>

- ### Verwendung von XFOIL und XROTOR
- ### Implementierung eines Python-Interface
    * ### Im Netz nur unbrauchbare oder schlecht dokumentierte Pakete...

#### Eigenes Interface:

<img src="figures/xsoftware.svg" alt="Xsoftware" title="" align="right"/>


```python
from util_loads import Xfoil

polar_file = '_xfoil_polar.txt'

# Das Inputfile wird mit einem Context-Manager generiert und automatisch abgeschickt:

with Xfoil() as x:
    x.run('aero')
    #...
    x.run('quit')
    
# Outputfiles werden mit statischen Methoden eingelesen

polar = Xfoil.read_polar(polar_file)
```

---

## Lastrechnungs-Package für Propeller:

<img src="figures/util_loads.svg" alt="util_loads" title="" align="right"/>

- ### Basis: XFOIL und XROTOR 
- ### Verwendung beliebiger Profile und Grundrisse
- ### Auswertung verschiedener Lastfälle und Bildung einhüllender Lastverläufe

In [ ]:
# Local imports
from util_loads import Propeller, Airfoil, Loadcase

airfoil = Airfoil('mf3218.xfo', 500000, iter_limit=600)

# Instantiate Propeller and assign geometry and airfoils

propeller = Propeller(number_of_blades=2,
                      tip_radius=0.412,
                      hub_radius=0.04,
                      )

propeller.geometry = np.array([[0.10,0.078,0],
                               [0.121, 0.078, 0.],
                               [0.155, 0.100, 5.99],
                               [0.223, 0.160, 17.97],
                               [0.345, 0.149, 14.44],
                               [0.417, 0.142, 12.68],
                               [0.490, 0.135, 11.18],
                               [0.563, 0.128, 9.94],
                               [0.636, 0.121, 8.97],
                               [0.709, 0.114, 8.26],
                               [0.782, 0.107, 7.81],
                               [0.854, 0.100, 7.63],
                               [0.947, 0.091, 7.5],
                               [1., 0.066, 7.5],                               
                               ])

propeller.sections = [[0.121, airfoil],
                      [1., airfoil],]

In [ ]:
airfoil.set_polar(alpha_start=-7, alpha_stop=20, alpha_inc=0.25)

In [ ]:
# Instantiate Loadcases

propeller.add_loadcase(loadcase=Loadcase(name='Max RPM', flight_speed=0.01))

propeller.loadcases[0][0].set_data('rpm',4000)